In [ ]:
import os
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np

map_folderdir = "./"
directory = os.listdir(map_folderdir)
flag = 0
for fname in directory:
    if "SLCmap" in fname:
        map_file = os.path.join(map_folderdir, fname)
        flag = 1

if flag == 0:
    errorMessage = 'Error: The file does not exist in the folder:\n ' + map_folderdir
    warnings.warn(errorMessage)

print('Now reading ' + map_file + "\n")
x = sio.loadmat(map_file)
map_struct = x['SLC']

# Define a new struct named SLC
SLC = map_struct[0][0]
column_map = dict(zip([name for name in SLC.dtype.names], [i for i in range(len(SLC.dtype.names))]))

dem = SLC[column_map["dem"]]
bldg = 0.3048 * SLC[column_map["hybrid_bldg"]]
map_ = SLC[column_map["dem"]] + 0.3048 * SLC[column_map["hybrid_bldg"]]

def replace_border_zeros(arr):
    rows, cols = arr.shape

    # Function to find nearest non-zero for a given position
    def nearest_non_zero(i, j):
        for d in range(1, max(rows, cols)):
            for di in range(-d, d + 1):
                for dj in range(-d, d + 1):
                    if 0 <= i + di < rows and 0 <= j + dj < cols and arr[i + di, j + dj] != 0:
                        return arr[i + di, j + dj]
        return 0  # In case there are no non-zero neighbors

    # Replace zeros on top and bottom rows
    for j in range(cols):
        if arr[0, j] == 0:
            arr[0, j] = nearest_non_zero(0, j)
        if arr[rows - 1, j] == 0:
            arr[rows - 1, j] = nearest_non_zero(rows - 1, j)

    # Replace zeros on left and right columns
    for i in range(rows):
        if arr[i, 0] == 0:
            arr[i, 0] = nearest_non_zero(i, 0)
        if arr[i, cols - 1] == 0:
            arr[i, cols - 1] = nearest_non_zero(i, cols - 1)

    return arr

# Example array

# Replace border zeros
dem = replace_border_zeros(dem)


In [ ]:
data_points = np.array([
    [966, 2992],
    [2569, 3767],
    [2873, 3447],
    [2621, 4286],
    [1312, 3830],
    [3828, 2667],
    [242, 2442],
    [1711, 3145],
    [2852, 1584],
    [1903, 2393]
])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage

x = np.linspace(0, map_.shape[1], map_.shape[1] , endpoint=False)
y = np.linspace(0, map_.shape[0], map_.shape[0], endpoint=False)
X, Y = np.meshgrid(x, y)


In [ ]:


# Perform edge detection using the Canny method
BW1 = ndimage.sobel(dem)

# Create an empty array of the same size as SLC.dem
edge_image_dem = np.zeros_like(dem)

# Place the edge detection result back into the array
edge_image_dem = BW1

# Perform edge detection using the Canny method
BW2 = ndimage.sobel(bldg)

# Create an empty array of the same size as SLC.dem
edge_image_bldg = np.zeros_like(bldg)

# Place the edge detection result back into the array
edge_image_bldg = BW2

In [ ]:
plt.figure(figsize=(11, 8))
plt.contourf(X, Y, np.abs(edge_image_dem), 100, cmap='hot')
plt.colorbar(label='Height')

plt.xlabel('UTM_E [m]')
plt.ylabel('UTM_N [m]')
plt.title('Edge Structure')
plt.show()

In [ ]:
# Plot the data collection points for reference


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
matplotlib.rcParams['font.family'] = 'Times New Roman'

# Example data for demonstration
# x = np.linspace(-5, 5, 100)
# y = np.linspace(-5, 5, 100)
# X, Y = np.meshgrid(x, y)
# Z1 = np.exp(-X**2 - Y**2)
# Z2 = np.cos(X) * np.sin(Y)

# Plotting
fig, ax = plt.subplots(figsize=(13, 7))
ax.set_aspect('equal')

# First dataset (e.g., edge_image_dem)
plt.contourf(X, Y, bldg, 100, cmap='gist_gray')
cbar1 = plt.colorbar(label='Building Height [m]')
cbar1.set_label('Building Height [m]', size=20)  # adjust the size value as needed
cbar1.ax.tick_params(labelsize=19)

# Second dataset overlay with colormap
contours = plt.contour(X, Y, dem, 10, cmap='gist_earth', linewidths=3.5)
manual_locations = [(2100, 1700), (4000, 2500), (4500, 4100), (500, 1000), (4000, 3500), (5200, 5200)]
plt.clabel(contours, inline=True, inline_spacing=13, fontsize=14, manual=manual_locations)
cbar2 = plt.colorbar(contours, label='Terrain Elevation [m]', location='left')
cbar2.set_label('Terrain Elevation [m]', size=20)  # adjust the size value as needed

cbar2.ax.tick_params(labelsize=19)

scatter = plt.scatter(data_points[:, 0], data_points[:, 1], c='red',  s=50, label='Data Collection Points')
plt.legend(loc='lower left')

N1 = map_.shape[0]
N2 = map_.shape[1]
en = SLC[column_map["axis"]]
UTM_long = np.linspace(en[0, 2], en[0, 3] - SLC[column_map["cellsize"]], N1)
UTM_lat = np.linspace(en[0, 0], en[0, 1] - SLC[column_map["cellsize"]], N2)
UTM_long = np.squeeze(UTM_long)
UTM_lat = np.squeeze(UTM_lat)
interval = 100
ax.set_xticks(range(0, len(UTM_lat), interval))
ax.set_xticklabels(UTM_lat[::interval], fontsize=14, rotation=0)

ax.set_yticks(range(0, len(UTM_long), interval))
ax.set_yticklabels(UTM_long[::interval], fontsize=14)

interval = max(1, len(UTM_lat) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_lat), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_lat[::interval]]

# Set the tick values and labels
plt.xticks(ticks=tick_values, labels=tick_labels, fontsize=14, rotation=0)

interval = max(1, len(UTM_long) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_long), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_long[::interval]]

# Adjust the overlapping xticks and yticks by setting the first ytick to be empty
plt.yticks(ticks=[0] + tick_values[1:], labels=[""] + tick_labels[1:], fontsize=14, rotation=90)

# Setting X-axis title
ax.set_xlabel('UTM$_E$ [m]', fontsize=24)
ax.set_ylabel('UTM$_N$ [m]', fontsize=24)
ax.xaxis.labelpad = 10  # title standoff


plt.show()


In [ ]:
dem[dem==0]

In [ ]:
import numpy as np

def replace_border_zeros(arr):
    rows, cols = arr.shape

    # Function to find nearest non-zero for a given position
    def nearest_non_zero(i, j):
        for d in range(1, max(rows, cols)):
            for di in range(-d, d + 1):
                for dj in range(-d, d + 1):
                    if 0 <= i + di < rows and 0 <= j + dj < cols and arr[i + di, j + dj] != 0:
                        return arr[i + di, j + dj]
        return 0  # In case there are no non-zero neighbors

    # Replace zeros on top and bottom rows
    for j in range(cols):
        if arr[0, j] == 0:
            arr[0, j] = nearest_non_zero(0, j)
        if arr[rows - 1, j] == 0:
            arr[rows - 1, j] = nearest_non_zero(rows - 1, j)

    # Replace zeros on left and right columns
    for i in range(rows):
        if arr[i, 0] == 0:
            arr[i, 0] = nearest_non_zero(i, 0)
        if arr[i, cols - 1] == 0:
            arr[i, cols - 1] = nearest_non_zero(i, cols - 1)

    return arr

# Example array

# Replace border zeros
dem = replace_border_zeros(dem)
bldg = replace_border_zeros(bldg)
